In [6]:
import pandas as pd
from geopy.distance import vincenty
import geohash
import numpy as np
import csv
import operator
import collections
import datetime
from datetime import datetime
from collections import defaultdict
from tqdm import tqdm_notebook

In [2]:
trial = pd.read_csv('pivot_table.csv')

In [3]:
piv = pd.pivot_table(trial, index='ts10', columns='ID' ,values = ['lat','lon','accuracy'])

In [4]:
piv = piv.asfreq('5Min', method=None)

In [5]:
def cij(piv, i,j, thrs=30):
    '''
    Generate the cij i and j
    '''
    #encounters
    i_lat = piv['lat'][i]
    i_long = piv['lon'][i]
    j_lat = piv['lat'][j]
    j_long = piv['lon'][j]
    i_point = zip(i_lat, i_long)
    j_point = zip(j_lat, j_long)
    dist = [vincenty(i_point[i], j_point[i]).meters  if pd.notnull(i_point[i][0]) and pd.notnull(j_point[i][0]) else 9999 for i in range(0, len(i_point))]
    encounters = [1 if x<=thrs else 0 for x in dist]
    return encounters

In [9]:
keys = trial.ID.unique()

import itertools
count = 0
distance = pd.DataFrame()
for subset in tqdm_notebook(itertools.combinations(keys, 2)):
    i = subset[0]
    j = subset[1]
    piv['{0}'.format(subset)] = cij(piv, i, j, 20)
    count = count + 1

In [ ]:
idx = piv.groupby(lambda x: x.time())["('1rishabhtrivedi', 'bhagyeshvikani')"].sum().index
idx.shape

In [ ]:
cij_frame = pd.DataFrame(index=idx)
for subset in itertools.combinations(keys, 2):
    subset = str(subset)
    cij_frame[subset] = piv.groupby(lambda x: x.time())[subset].sum()

In [ ]:
div_frame = pd.DataFrame(index=['div'])
def diversity():
    for subset in tqdm_notebook(itertools.combinations(keys, 2)):
        subset = str(subset)

        vector = cij_frame[subset]      
        p_vector = vector/float(sum(vector))        
        plogged = p_vector.apply(lambda x: x*pd.np.log(x))
        entropy = -1*pd.np.nansum(plogged)
        div_score = math.exp(entropy)
        div_frame[subset] = div_score
diversity()

In [ ]:
div_frame.T.describe()

In [ ]:
df2 = div_frame.T.reset_index()
df2['Person 1'] = '' 

In [ ]:
def custom(x):
    try:
        return x.replace('(','').replace(')','').replace("'",'')
    except ValueError:
        print(x)

In [ ]:
df2['Person_1'] = df2['Person_1'].map(custom)
df2['Person_2'] = df2['Person_2'].map(custom)

In [ ]:
df2.to_csv('Diversity_Scores.csv')